# Water Treatment Data Analysis

I'm currently workign through a graciously donated data-set from UCI [https://archive.ics.uci.edu/ml/datasets/Water+Treatment+Plant], consisting of operating data from a water treatment plant located in Madrid Spain. This analysis is strictly for the purposes of curiosity and honing my skills with new tools. 

This document represents the current state of analysis and my thought process. This document will live until I can find some reasonable (and interesting) conclusions, which will be summarized in a separate report. 

Thanks for joining me.

In [1]:
#importing pandas, numpy, setting matplot to show inline
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#bringing in data, setting up names of columns based on names from UCI.
#NA values were represented by ? per the instructions accompanying the dataset. 
#Using dates as index
df = pd.read_csv('water-treatment.csv', header=None, na_values=["?"],index_col=0)
#taking a look at the initial dataset. 
df.head(10)


,1,2,3,4,5,6,7,8,9,10,...,29,30,31,32,33,34,35,36,37,38
0,,,,,,,,,,,,,,,,,,,,,
D-1/3/90,44101.0,1.5,7.8,NaN,407.0,166.0,66.3,4.5,2110,7.9,...,2000.0,NaN,58.8,95.5,NaN,70.0,NaN,79.4,87.3,99.6
D-2/3/90,39024.0,3.0,7.7,NaN,443.0,214.0,69.2,6.5,2660,7.7,...,2590.0,NaN,60.7,94.8,NaN,80.8,NaN,79.5,92.1,100.0
D-4/3/90,32229.0,5.0,7.6,NaN,528.0,186.0,69.9,3.4,1666,7.7,...,1888.0,NaN,58.2,95.6,NaN,52.9,NaN,75.8,88.7,98.5
D-5/3/90,35023.0,3.5,7.9,205.0,588.0,192.0,65.6,4.5,2430,7.8,...,1840.0,33.1,64.2,95.3,87.3,72.3,90.2,82.3,89.6,100.0
D-6/3/90,36924.0,1.5,8.0,242.0,496.0,176.0,64.8,4.0,2110,7.9,...,2120.0,NaN,62.7,95.6,NaN,71.0,92.1,78.2,87.5,99.5
D-7/3/90,38572.0,3.0,7.8,202.0,372.0,186.0,68.8,4.5,1644,7.8,...,1764.0,NaN,59.7,96.5,86.7,78.3,90.1,73.1,84.9,100.0
D-8/3/90,41115.0,6.0,7.8,NaN,552.0,262.0,64.1,5.0,1603,7.8,...,1703.0,NaN,61.9,93.8,89.1,79.8,NaN,86.2,90.1,99.0
D-9/3/90,36107.0,5.0,7.7,215.0,489.0,334.0,40.7,6.0,1613,7.6,...,1606.0,NaN,70.4,95.6,90.6,53.7,92.1,66.9,94.6,100.0
D-11/3/90,29156.0,2.5,7.7,206.0,451.0,194.0,69.1,4.5,1249,7.7,...,1338.0,46.1,43.6,92.5,85.6,58.2,92.2,73.8,90.2,99.4


The data has a lot of holes in it. I haven't decided what do about those yet. We'll have to explore any accompanying details for why things are missing. There are a vareity of options, including filling the data with the mean for that column, by interpolating over the next two points, or by leaving it out completely. Once some questions have been identified, I'll investigate the best method for handling this data. 

In [2]:
df.isna().sum() #determinig how much data we're missing on each entry

1     18
2      3
3      0
4     23
5      6
6      1
7     11
8     25
9      0
10     0
11    40
12     0
13    11
14    24
15     0
16     0
17    28
18     9
19     2
20    13
21    25
22     0
23     1
24    23
25    18
26     5
27    17
28    28
29     1
30    62
31     4
32    27
33    40
34    26
35    36
36    25
37     8
38    31
dtype: int64

In [3]:
#Figuring out names and a description, based on supplied data. This list was doctored in a text editor before I brought it in here.
names = pd.read_csv('property_names.csv', index_col=0, names = ['#','Property Name','Description'])
names

,Property Name,Description
#,,
1,Q-E,(input flow to plant)
2,ZN-E,(input Zinc to plant)
3,PH-E,(input pH to plant)
4,DBO-E,(input Biological demand of oxygen to plant)
5,DQO-E,(input chemical demand of oxygen to plant)
6,SS-E,(input suspended solids to plant)
7,SSV-E,(input volatile supended solids to plant)
8,SED-E,(input sediments to plant)
9,COND-E,(input conductivity to plant)


In [ ]:
sns.pairplot(df); # This is currently taking too long and hanging.
# need to find another initial data comparison method

C:\Users\spenc\Anaconda3\envs\py3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial)
C:\Users\spenc\Anaconda3\envs\py3\lib\site-packages\numpy\core\_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)


ValueError: max must be larger than min in range parameter.